In [65]:
import tensorflow as tf
import numpy as np
import scipy.misc
import tensorflow.keras.layers as tfl

In [ ]:
tf.config.run_functions_eagerly(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    print(gpu)

In [32]:
def downsampling_block(inputs=None, n_filters=32, dropout_prob=0, batchnorm=True):
    """
    Convolutional downsampling block
    
    Arguments:
        inputs -- Input tensor
        n_filters -- Number of filters for the convolutional layers
        dropout_prob -- Dropout probability
        batchnorm -- Boolean determining if batchnorm is applied after the Conv2D layer
    Returns: 
        conv --  Tensor output
    """

    conv = tfl.Conv2D(n_filters,
                4,
                strides=(2,2),
                padding='same',
                kernel_initializer=tf.random_normal_initializer(0., 0.02),
                use_bias=False)(inputs)
    
    if batchnorm:
        conv = tfl.BatchNormalization()(conv)

    if dropout_prob > 0:
        conv = tfl.Dropout(dropout_prob)(conv)

    conv = tfl.LeakyReLU()(conv)
    
    return conv

In [33]:
def upsampling_block(expansive_input, contractive_input, n_filters=32, dropout_prob=0, batchnorm=True):
    """
    Convolutional upsampling block
    
    Arguments:
        expansive_input -- Input tensor from previous layer
        contractive_input -- Input tensor from previous skip layer
        n_filters -- Number of filters for the convolutional layers
        dropout_prob -- Probability for Dropout in the Conv2DTranspose layer
        batchnorm -- Boolean determining if batchnorm is applied after the Conv2DTranspose layer
    Returns: 
        conv -- Tensor output
    """

    conv = tfl.Conv2DTranspose(
                n_filters,
                4,
                strides=(2,2),
                padding='same',
                kernel_initializer=tf.random_normal_initializer(0., 0.02),
                use_bias=False)(expansive_input)
        
    if batchnorm:
        conv = tfl.BatchNormalization()(conv)

    if dropout_prob > 0:
        conv = tfl.Dropout(dropout_prob)(conv)

    conv = tfl.Concatenate(axis=3)([conv, contractive_input])


    conv = tfl.ReLU()(conv)
    
    return conv

In [46]:
def Unet(input_shape=[None,None,3], output_channels=1):
    """
    Stage-wise implementation of the architecture of the U-net model.

    Arguments:
    input_shape -- shape of the images of the dataset
    output_channels -- amount of output channels

    Returns:
    model -- a Model instance in Keras
    """
    
    X_input = tfl.Input(input_shape)
    skips = []
    
    X = downsampling_block(X_input, 64, batchnorm=False)
    skips.append(X)
    X = downsampling_block(X, 128)
    skips.append(X)
    X = downsampling_block(X, 256)
    skips.append(X)
    X = downsampling_block(X, 512)
    skips.append(X)
    X = downsampling_block(X, 512)
    skips.append(X)
    X = downsampling_block(X, 512)
    skips.append(X)
    X = downsampling_block(X, 512)
    skips.append(X)
    X = downsampling_block(X, 512)
    skips.append(X)

    X = upsampling_block(X, skips[-2], 512, dropout_prob=0.5)
    X = upsampling_block(X, skips[-3], 512, dropout_prob=0.5)
    X = upsampling_block(X, skips[-4], 512, dropout_prob=0.5)
    X = upsampling_block(X, skips[-5], 512)
    X = upsampling_block(X, skips[-6], 256)
    X = upsampling_block(X, skips[-7], 128)
    X = upsampling_block(X, skips[-8], 64)

    X = tfl.Conv2DTranspose(output_channels, 4,
                            strides=2,
                            padding='same',
                            kernel_initializer=tf.random_normal_initializer(0., 0.02),
                            activation='tanh')(X)

    model = tf.keras.Model(inputs = X_input, outputs = X)

    return model

Generator = Unet()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_26 (Conv2D)             (None, None, None,   3072        ['input_3[0][0]']                
                                64)                                                               
                                                                                                  
 leaky_re_lu_24 (LeakyReLU)     (None, None, None,   0           ['conv2d_26[0][0]']              
                                64)                                                         

In [47]:
def PatchGAN(input_shape=[256, 256, 3], output_channels=1):
    """
    Stage-wise implementation of the architecture of the PatchGAN model.

    Arguments:
    input_shape -- shape of the images of the dataset
    output_channels -- amount of output channels

    Returns:
    model -- a Model instance in Keras
    """

    initializer = tf.random_normal_initializer(0., 0.02)

    inp = tfl.Input(shape=input_shape, name='input_image')
    tar = tfl.Input(shape=[input_shape[0], input_shape[1], output_channels], name='target_image')

    X = tfl.Concatenate()([inp, tar])

    X = downsampling_block(X, 64, batchnorm=False)
    X = downsampling_block(X, 128)
    X = downsampling_block(X, 256)

    X = tfl.ZeroPadding2D()(X)

    X = tfl.Conv2D(512, 4, strides=1, kernel_initializer=initializer, use_bias=False)(X)

    X = tfl.BatchNormalization()(X)

    X = tfl.LeakyReLU()(X)

    X = tfl.ZeroPadding2D()(X)

    X = tfl.Conv2D(1, 4, strides=1, kernel_initializer=initializer)(X)

    return tf.keras.Model(inputs=[inp, tar], outputs=X)

Discriminator = PatchGAN()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 target_image (InputLayer)      [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 concatenate_23 (Concatenate)   (None, 256, 256, 4)  0           ['input_image[0][0]',            
                                                                  'target_image[0][0]']     

In [48]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(disc_real_output, disc_generated_output):
    """
    Discriminator loss for the PatchGAN.

    Arguments:
    disc_real_output -- output of the discriminator on real elevation maps
    dis_generated_output -- output of the discriminator on generated maps

    Returns:
    real_loss + generated_loss -- overall loss of the discriminator
    """
    
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

    return real_loss + generated_loss

def generator_loss(disc_generated_output, gen_output, target):
    """
    Generator loss for the Unet.

    Arguments:
    disc_generated_output -- output of the discriminator on generated maps
    gen_output -- output of the generator on the sample
    target -- real elevation map of the sample

    Returns:
    total_gen_loss -- overall loss of the generator
    gan_loss -- loss due to not fooling the discriminator
    l1_loss -- loss due to difference to target
    """

    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

    total_gen_loss = gan_loss + (100 * l1_loss)

    return total_gen_loss, gan_loss, l1_loss

generator_optimizer = tf.keras.optimizers.Adam(0, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(0, beta_1=0.5) #2e-4

In [49]:
@tf.function
def train_step(input_image, target):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_output = Generator(input_image, training=True)
        disc_real_output = Discriminator([input_image, target], training=True)

        disc_generated_output = Discriminator([input_image, gen_output], training=True)

        gen_loss, gan_loss, l1_loss = generator_loss(disc_generated_output, gen_output, target)
        disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

    generator_gradients = gen_tape.gradient(gen_loss, Generator.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss, Discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_gradients, Generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, Discriminator.trainable_variables))
    
    return gen_loss, gan_loss, l1_loss, disc_loss

In [50]:
import csv

def savedata(dataname, *data):
    with open(dataname, 'a') as fd:
        writer = csv.writer(fd)
        writer.writerow(list(data))

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, images, labels, batch_size=128, shuffle=True):
    super().__init__()
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.shuffle = shuffle
    key_array = []
    self.key_array = np.arange(self.images.shape[0], dtype=np.uint32)
    self.on_epoch_end()

  def __len__(self):
    return len(self.key_array)//self.batch_size

  def __getitem__(self, index):
    keys = self.key_array[index*self.batch_size:(index+1)*self.batch_size]
    x = np.asarray(self.images[keys], dtype=np.float32)
    y = np.asarray(self.labels[keys], dtype=np.float32)
    return x, y

  def on_epoch_end(self):
    if self.shuffle:
      self.key_array = np.random.permutation(self.key_array)

In [51]:
import time

def fit(data_generator, num_epochs):
    epoch_times = []
    num_batches = len(data_generator)
    for epoch in range(num_epochs):
        start = time.time()

        l1_cost = []
        generator_cost = []
        gan_cost = []
        discriminator_cost = []
        for batch in range(num_batches):
            x, y = data_generator[batch]
            gen_loss, gan_loss, l1_loss, disc_loss = train_step(x, y)
            l1_cost.append(l1_loss)
            generator_cost.append(gen_loss)
            gan_cost.append(gan_loss)
            discriminator_cost.append(disc_loss)
            
        data_generator.on_epoch_end()

        l1_cost = np.mean(l1_cost)
        generator_cost = np.mean(generator_cost)
        gan_cost = np.mean(gan_cost)
        discriminator_cost = np.mean(discriminator_cost)
      
        savedata("./training_data.csv", epoch, l1_cost, generator_cost, gan_cost, discriminator_cost)

        epoch_times.append(time.time()-start)

        #if (epoch + 1) % 20 == 0 or epoch == 0:
            #tf.saved_model.save(Generator, './models/')
            #print('Model saved')

        print('Time taken for epoch {} is {} seconds. \n'.format(epoch + 1, epoch_times[-1]))

        print('Estimated Time left for remaining {} epochs is {:.2f} sec = {:.2f} hours\\n'.format(num_epochs - epoch - 1, 
        (num_epochs - epoch - 1) * sum(epoch_times) / len(epoch_times), (num_epochs - epoch - 1) * sum(epoch_times) / len(epoch_times)/3600))

In [60]:
image1 = tf.keras.preprocessing.image.load_img("../test_image_5.png", target_size=(256,256), keep_aspect_ratio=True)
image1 = tf.keras.preprocessing.image.img_to_array(image1)
image2 = tf.keras.preprocessing.image.load_img("../test_image_3.png", target_size=(256,256), keep_aspect_ratio=True)
image2 = tf.keras.preprocessing.image.img_to_array(image2)
input_arr = tf.convert_to_tensor([[image1,image1,image2,image2]])
input_arr = (input_arr / 127.5) - 1
output_arr = tf.random.normal((1,4,256,256,1))
dataset = zip(input_arr, output_arr)